### (学習)YAMNetで特徴量抽出→特徴量とラベルのデータセット追加→音楽属性推論モデルの作成

In [ ]:
#APIのセットアップ
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'

credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)

#音楽データの取得
def get_track_preview_url(track_id):
    track_info = sp.track(track_id)
    return track_info['preview_url']

# トラックIDの例
track_id = 'YOUR_TRACK_ID'
preview_url = get_track_preview_url(track_id)
print(preview_url)

#音楽の属性の取得
def get_track_attributes(track_id):
    features = sp.audio_features([track_id])[0]
    return {
        'danceability': features['danceability'],
        'energy': features['energy'],
        'valence': features['valence']
        # 他の属性も必要に応じて追加
    }
# トラックIDの例
track_id = 'YOUR_TRACK_ID'
track_attributes = get_track_attributes(track_id)
print(track_attributes)

In [ ]:
#YAMNetで特徴量抽出
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from scipy.io import wavfile
import requests

# YAMNetモデルのロード
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

def download_and_predict_features(url):
    response = requests.get(url)
    with open('temp_music_preview.wav', 'wb') as f:
        f.write(response.content)
    
    # WAVファイルの読み込み
    sample_rate, wav_data = wavfile.read('temp_music_preview.wav', 'rb')
    waveform = np.mean(wav_data, axis=1)  # ステレオからモノラルに変換
    waveform = waveform / tf.int16.max  # 正規化

    # 特徴量の抽出
    _, embeddings, _ = yamnet_model(waveform)
    return embeddings.numpy()

# プレビューURLから特徴量を抽出
features = download_and_predict_features(preview_url)
print(features)

In [ ]:
#特徴量とラベルのデータセット作成
import numpy as np

# 特徴量とラベルのデータセットを初期化
features_dataset = []
labels_dataset = []

# 複数のトラックIDに対して特徴量とラベルを取得
track_ids = ['TRACK_ID_1', 'TRACK_ID_2', 'TRACK_ID_3']  # 例として3つのトラックIDを使用する場合
for track_id in track_ids:
    # Spotify APIから音楽属性を取得
    track_attributes = get_track_attributes(track_id)
    
    # プレビューURLから特徴量を抽出
    preview_url = get_track_preview_url(track_id)
    features = download_and_predict_features(preview_url)

    # データセットに追加
    features_dataset.append(features)
    labels_dataset.append(track_attributes)

# NumPy配列に変換
features_dataset = np.array(features_dataset)
labels_dataset = np.array(labels_dataset)

# データセットの確認
print(features_dataset.shape)
print(labels_dataset.shape)

In [ ]:
#音楽属性の推論モデルの訓練
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump

# データセットの分割（訓練データとテストデータ）
X_train, X_test, y_train, y_test = train_test_split(features_dataset, labels_dataset, test_size=0.3, random_state=42)

# ランダムフォレスト分類器の訓練
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
#モデルの保存
dump(model, 'music_recommendation_model.joblib')

# モデルの評価
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

### 環境音→音楽属性を推論→類似度から音楽を推薦

In [ ]:
from joblib import load

# モデルの読み込み
model = load('music_recommendation_model.joblib')

# 入力音の特徴量抽出
input_audio_features = download_and_predict_features(input_audio_url)

# 音楽属性の推論
predicted_attributes = model.predict([input_audio_features])[0]

# Spotify APIを使用して音楽を検索し、推薦する関数
def recommend_music(sp, music_attributes, limit=10):
    """
    Spotify APIを使用して音楽を検索し、推薦する関数。類似度スコアを含む。
    """
    # マッピングされた音楽属性をクエリに変換
    query = ' '.join(f'{key}:{value}' for key, value in music_attributes.items())

    # Spotify APIで音楽を検索
    results = sp.search(q=query, limit=limit, type='track')

    # 検索結果からトラック情報を抽出
    tracks = results['tracks']['items']
    recommended_tracks = []
    for track in tracks:
        # トラックの音楽属性を取得
        track_features = sp.audio_features(track['id'])[0]

        # 類似度スコアの計算（ユークリッド距離の逆数を使用）
        similarity_score = 1 / (1 + sum((music_attributes[key] - track_features[key])**2 for key in music_attributes))

        track_info = {
            'id': track['id'],
            'name': track['name'],
            'artists': ', '.join(artist['name'] for artist in track['artists']),
            'url': track['external_urls']['spotify'],
            'similarity_score': similarity_score
        }
        recommended_tracks.append(track_info)

    # 類似度スコアでソートして返す
    recommended_tracks.sort(key=lambda x: x['similarity_score'], reverse=True)
    return recommended_tracks


# Spotify APIを使用して音楽を検索し、推薦
recommended_tracks = recommend_music(sp, predicted_attributes, limit=5)

# 推薦されたトラックの表示
for track in recommended_tracks:
    print(f"{track['name']} by {track['artists']} - {track['url']} (Similarity: {track['similarity_score']:.2f})")
